In [8]:
import pandas as pd
import os

# CSV 불러오기
file_path = r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250904_다 찍기\csv파일\전압 전류 전하\V_distance.csv"
df = pd.read_csv(file_path, header=None)

# 첫 번째 열 제외하고 나머지 열에 10 곱하기
df.iloc[:, 1:] = df.iloc[:, 1:] * 10

# 저장 경로 만들기
folder = os.path.dirname(file_path)                        # 원래 폴더
filename = os.path.basename(file_path)                     # 원래 파일명
output_path = os.path.join(folder, "scaled_" + filename)   # scaled_ 붙이기

# 결과 저장
df.to_csv(output_path, index=False, header=False, float_format="%.6f")

print(f"Saved scaled CSV to: {output_path}")


Saved scaled CSV to: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250904_다 찍기\csv파일\전압 전류 전하\scaled_V_distance.csv
